# Library import 및 Spark 세션 생성

In [19]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder \
    .appName("nba-shot-logs") \
    .getOrCreate()

# Data Table 생성

In [28]:
df = pd.read_csv('./data/shot_logs.csv')
print(df.columns)

nba_df = spark.createDataFrame(df)
nba_df.createOrReplaceTempView("nba")

Index(['GAME_ID', 'MATCHUP', 'LOCATION', 'W', 'FINAL_MARGIN', 'SHOT_NUMBER',
       'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME',
       'SHOT_DIST', 'PTS_TYPE', 'SHOT_RESULT', 'CLOSEST_DEFENDER',
       'CLOSEST_DEFENDER_PLAYER_ID', 'CLOSE_DEF_DIST', 'FGM', 'PTS',
       'player_name', 'player_id'],
      dtype='object')


# SQL 정복하기

## 전체 개수 구하기

In [21]:
result_df = spark.sql("""
    select count(*)
    from nba
""")
result_df.show()

23/02/26 15:25:23 WARN TaskSetManager: Stage 18 contains a task of very large size (1182 KiB). The maximum recommended task size is 1000 KiB.
+--------+
|count(1)|
+--------+
|  128069|
+--------+



## GAME_ID로 group by 하기

In [40]:
result_df = spark.sql("""
    select GAME_ID, MATCHUP, LOCATION, W
    from nba
    where GAME_ID in ('21400899', '21400890', '21400882')
    group by GAME_ID, MATCHUP, LOCATION, W
    order by GAME_ID
""")
result_df.show()

23/02/26 15:46:48 WARN TaskSetManager: Stage 48 contains a task of very large size (1182 KiB). The maximum recommended task size is 1000 KiB.
+--------+--------------------+--------+---+
| GAME_ID|             MATCHUP|LOCATION|  W|
+--------+--------------------+--------+---+
|21400882|MAR 01, 2015 - OR...|       H|  L|
|21400882|MAR 01, 2015 - CH...|       A|  W|
|21400890|MAR 03, 2015 - LA...|       A|  L|
|21400890|MAR 03, 2015 - CH...|       H|  W|
|21400899|MAR 04, 2015 - CH...|       A|  W|
|21400899|MAR 04, 2015 - BK...|       H|  L|
+--------+--------------------+--------+---+

